In [4]:
import numpy as np
from scipy import linalg, optimize

MAX_ITER = 100


def group_lasso(X, y, alpha, groups, max_iter=MAX_ITER, rtol=1e-6,
             verbose=False):
    """
    Linear least-squares with l2/l1 regularization solver.
    Solves problem of the form:
               .5 * |Xb - y| + n_samples * alpha * Sum(w_j * |b_j|)
    where |.| is the l2-norm and b_j is the coefficients of b in the
    j-th group. This is commonly known as the `group lasso`.
    Parameters
    ----------
    X : array of shape (n_samples, n_features)
        Design Matrix.
    y : array of shape (n_samples,)
    alpha : float or array
        Amount of penalization to use.
    groups : array of shape (n_features,)
        Group label. For each column, it indicates
        its group apertenance.
    rtol : float
        Relative tolerance. ensures ||(x - x_) / x_|| < rtol,
        where x_ is the approximate solution and x is the
        true solution.
    Returns
    -------
    x : array
        vector of coefficients
    References
    ----------
    "Efficient Block-coordinate Descent Algorithms for the Group Lasso",
    Qin, Scheninberg, Goldfarb
    """

    # .. local variables ..
    X, y, groups, alpha = map(np.asanyarray, (X, y, groups, alpha))
    if len(groups) != X.shape[1]:
        raise ValueError("Incorrect shape for groups")
    w_new = np.zeros(X.shape[1], dtype=X.dtype)
    alpha = alpha * X.shape[0]

    # .. use integer indices for groups ..
    group_labels = [np.where(groups == i)[0] for i in np.unique(groups)]
    print group_labels
    H_groups = [np.dot(X[:, g].T, X[:, g]) for g in group_labels]
    print H_groups.__sizeof__(), H_groups
    eig = map(linalg.eigh, H_groups)
    Xy = np.dot(X.T, y)
    initial_guess = np.zeros(len(group_labels))

    def f(x, qp2, eigvals, alpha):
        return 1 - np.sum( qp2 / ((x * eigvals + alpha) ** 2))
    def df(x, qp2, eigvals, penalty):
        # .. first derivative ..
        return np.sum((2 * qp2 * eigvals) / ((penalty + x * eigvals) ** 3))

    if X.shape[0] > X.shape[1]:
        H = np.dot(X.T, X)
    else:
        H = None

    for n_iter in range(max_iter):
        w_old = w_new.copy()
        for i, g in enumerate(group_labels):
            # .. shrinkage operator ..
            eigvals, eigvects = eig[i]
            w_i = w_new.copy()
            w_i[g] = 0.
            if H is not None:
                X_residual = np.dot(H[g], w_i) - Xy[g]
            else:
                X_residual = np.dot(X.T, np.dot(X[:, g], w_i)) - Xy[g]
            qp = np.dot(eigvects.T, X_residual)
            if len(g) < 2:
                # for single groups we know a closed form solution
                w_new[g] = - np.sign(X_residual) * max(abs(X_residual) - alpha, 0)
            else:
                if alpha < linalg.norm(X_residual, 2):
                    initial_guess[i] = optimize.newton(f, initial_guess[i], df, tol=.5,
                                args=(qp ** 2, eigvals, alpha))
                    w_new[g] = - initial_guess[i] * np.dot(eigvects /  (eigvals * initial_guess[i] + alpha), qp)
                else:
                    w_new[g] = 0.


        # .. dual gap ..
        max_inc = linalg.norm(w_old - w_new, np.inf)
        if True: #max_inc < rtol * np.amax(w_new):
            residual = np.dot(X, w_new) - y
            group_norm = alpha * np.sum([linalg.norm(w_new[g], 2)
                         for g in group_labels])
            if H is not None:
                norm_Anu = [linalg.norm(np.dot(H[g], w_new) - Xy[g]) \
                           for g in group_labels]
            else:
                norm_Anu = [linalg.norm(np.dot(H[g], residual)) \
                           for g in group_labels]
            if np.any(norm_Anu > alpha):
                nnu = residual * np.min(alpha / norm_Anu)
            else:
                nnu = residual
            primal_obj =  .5 * np.dot(residual, residual) + group_norm
            dual_obj   = -.5 * np.dot(nnu, nnu) - np.dot(nnu, y)
            dual_gap = primal_obj - dual_obj
            if verbose:
                print 'Relative error: %s' % (dual_gap / dual_obj)
            if np.abs(dual_gap / dual_obj) < rtol:
                break

    return w_new


def check_kkt(A, b, x, penalty, groups):
    """Check KKT conditions for the group lasso
    Returns True if conditions are satisfied, False otherwise
    """
    group_labels = [groups == i for i in np.unique(groups)]
    penalty = penalty * A.shape[0]
    z = np.dot(A.T, np.dot(A, x) - b)
    safety_net = 1e-1 # sort of tolerance
    for g in group_labels:
        if linalg.norm(x[g]) == 0:
            if not linalg.norm(z[g]) < penalty + safety_net:
                return False
        else:
            w = - penalty * x[g] / linalg.norm(x[g], 2)
            if not np.allclose(z[g], w, safety_net):
                return False
    return True


FISTA + Group Lasso L2 with variable size of groups
mean loss: 0.475944, mean relative duality_gap: 0.000113, number of iterations: 20.000000


In [8]:
from sklearn import datasets
diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target
alpha = .1
groups = np.r_[[11, 11], np.arange(X.shape[1] - 2)]
coefs = group_lasso(X, y, alpha, groups, verbose=True)
print 'KKT conditions verified:', check_kkt(X, y, coefs, alpha, groups)

Relative error: 2.33177606485
Relative error: 0.473190450119
Relative error: 0.126825121057
Relative error: 0.0217456557803
Relative error: 0.0321324913375
Relative error: 0.00770328777563
Relative error: 0.00530338779656
Relative error: 0.00678894481934
Relative error: 0.00744198839788
Relative error: 0.00737399150524
Relative error: 0.00703761753172
Relative error: 0.00669235328303
Relative error: 0.00640758310735
Relative error: 0.00569487768198
Relative error: 0.00112936068656
Relative error: 0.000335788264732
Relative error: 9.35987100447e-05
Relative error: 3.81131188497e-05
Relative error: 1.59188155948e-05
Relative error: 5.42017651034e-06
Relative error: 1.3519845247e-06
Relative error: 3.5369116917e-07
KKT conditions verified:

NameError: name 'check_kkt' is not defined

In [13]:
print X.shape
print y.shape
print coefs.shape

(442, 10)
(442,)
(10,)


In [16]:
groups

array([11, 11,  0,  1,  2,  3,  4,  5,  6,  7])

In [12]:
from src.models.GroupLasso import group_lasso
import spams
import numpy as np

myfloat = np.float
param = {'numThreads' : -1,'verbose' : True,
         'lambda1' : 0.05, 'it0' : 10, 'max_it' : 200,
         'L0' : 0.1, 'tol' : 1e-3, 'intercept' : False,
         'pos' : False}

np.random.seed(0)
X = diabetes.data
X = np.asfortranarray(X, dtype=myfloat)
m = X.shape[0]
n = X.shape[1]

Y = np.asfortranarray(np.random.normal(size=(m, 1)))
Y = np.asfortranarray(Y - np.tile(np.mean(Y, 0), (Y.shape[0], 1)), dtype=myfloat)
Y = spams.normalize(Y)

Y = np.asfortranarray(np.ceil(5 * np.random.random(size=(m, Y.shape[1]))) - 1, dtype=myfloat)

nclasses = len(np.unique(Y))
W0 = np.zeros((X.shape[1], nclasses), dtype=myfloat, order="F")

print "X", type(X), X.shape
print "Y", type(Y), Y.shape, np.unique(Y)
print "nclasses", nclasses
print "W0", type(W0), W0.shape

print '\nMulti-logistic + Group Lasso L2 with variable size of groups'
param['loss'] = 'multi-logistic'
param['regul'] = 'l1'
param2 = param.copy()
# param2['groups'] = np.array(np.random.random_integers(1, 5, X.shape[1]), dtype=np.int32)
# param2['groups'] = np.array(np.arange(1881)+1, dtype=np.int32)
param2['lambda1'] = 0.01
print param2
W, optim_info = spams.fistaFlat(Y, X, W0, True, **param2)

X <type 'numpy.ndarray'> (442, 10)
Y <type 'numpy.ndarray'> (442, 1) [ 0.  1.  2.  3.  4.]
nclasses 5
W0 <type 'numpy.ndarray'> (10, 5)

Multi-logistic + Group Lasso L2 with variable size of groups
{'loss': 'multi-logistic', 'regul': 'l1', 'verbose': True, 'numThreads': -1, 'max_it': 200, 'pos': False, 'intercept': False, 'L0': 0.1, 'tol': 0.001, 'lambda1': 0.01, 'it0': 10}


In [13]:
print "W", W.shape
print "nonzero", len(W.nonzero()[0])

W (10, 5)
nonzero 0
